In [1]:
import scanpy as sc
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import doubletdetection
from scipy.stats import median_abs_deviation as mad
import numpy as np

In [2]:
#connect env R to env python 
os.environ['R_HOME'] = r"/home/ybeaumatin/anaconda3/envs/annot/lib/R"
os.environ['R_LIBS'] = r"/home/ybeaumatin/anaconda3/envs/annot/lib/R/library"

print("R_HOME:", os.environ['R_HOME'])
print("R_LIBS:", os.environ['R_LIBS'])

R_HOME: /home/ybeaumatin/anaconda3/envs/annot/lib/R
R_LIBS: /home/ybeaumatin/anaconda3/envs/annot/lib/R/library


In [3]:
import rpy2.robjects as ro
print(ro.r("R.version.string"))

[1] "R version 4.3.3 (2024-02-29)"



In [4]:
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_367528/3964782999.py:9: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [5]:
%%R
cat("hello world")

hello world

ValueError: Not an rpy2 R object and unable to map it to one: np.str_('value')

In [6]:
%%R
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

ValueError: Not an rpy2 R object and unable to map it to one: np.str_('value')

In [12]:
import warnings 
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [14]:
path_filtered_files_h5 = '/home/ybeaumatin/Documents/Data processing pipeline/scRNA/Data/h5_files_filtered'

In [15]:
adatas = [x for x in os.listdir(path_filtered_files_h5 ) 
          if x.endswith('.h5')]

In [16]:
def load_it(adata):
    samp= adata.split('_')[5]
    dpi_end = adata.split('_')[6]
    dpi = dpi_end.split('.')[0]
    adata = sc.read_10x_h5(path_filtered_files_h5 + '/' + adata)
    adata.obs['Sample'] = samp
    adata.obs['dpi'] = dpi
    adata.obs['Id'] = adata.obs['Sample'] + '_' + adata.obs['dpi']
    adata.obs.index = adata.obs.index + '-' + samp + '_' + dpi
    remove = ['Sample','dpi']

    adata.obs = adata.obs[[x for x in adata.obs.columns if x not in remove]]
    return adata

In [17]:
adatas = [load_it(ad) for ad in adatas]

In [18]:
adatas

[AnnData object with n_obs × n_vars = 9137 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome',
 AnnData object with n_obs × n_vars = 10542 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome',
 AnnData object with n_obs × n_vars = 8525 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome',
 AnnData object with n_obs × n_vars = 11004 × 18087
     obs: 'Id'
     var: 'gene_ids', 'feature_types', 'genome']

In [19]:
for ad in adatas:
    ad.var_names_make_unique()

In [20]:
def qc(adata):
    sc.pp.filter_cells(adata, min_genes = 200)
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    adata.var["ribo"] = adata.var_names.str.startswith("RPS", "RPL")
    adata.var["hb"] = adata.var_names.str.startswith("^HB[^(P)]")
    sc.pp.calculate_qc_metrics(adata,qc_vars=["mt","ribo","hb"], inplace = True, percent_top = [20], log1p=True)

    remove = ['total_counts_mt', 'log1p_total_counts_mt', 'total_counts_ribo',
              'log1p_total_counts_ribo','total_counts_hb','log1p_total_counts_hb']

    adata.obs = adata.obs[[x for x in adata.obs.columns if x not in remove]]
    return adata

In [21]:
adatas = [qc(ad) for ad in adatas]

In [22]:
adatas[0].obs

,Id,n_genes,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb
AAACAAGCAGCTCGCTATGTTGAC-1-Inf_J6,Inf_J6,4730,4730,8.461892,14274.0,9.566265,18.775396,0.175144,0.0,0.0
AAACCAATCACCTAATATGTTGAC-1-Inf_J6,Inf_J6,8415,8415,9.037890,54307.0,10.902427,19.159593,0.252270,0.0,0.0
AAACCAATCATTATGCATGTTGAC-1-Inf_J6,Inf_J6,5073,5073,8.531885,18919.0,9.847975,29.494159,0.190285,0.0,0.0
AAACCAATCATTGCATATGTTGAC-1-Inf_J6,Inf_J6,5705,5705,8.649274,20530.0,9.929691,16.585485,0.998539,0.0,0.0
AAACCAATCCAAATTCATGTTGAC-1-Inf_J6,Inf_J6,5398,5398,8.593969,14307.0,9.568574,6.304606,0.146781,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
TTTGGCGGTAACTACCATGTTGAC-1-Inf_J6,Inf_J6,2631,2631,7.875499,4842.0,8.485290,15.386204,0.309789,0.0,0.0
TTTGGCGGTAAGCGTAATGTTGAC-1-Inf_J6,Inf_J6,4769,4769,8.470102,15426.0,9.643874,15.493323,0.187994,0.0,0.0
TTTGGCGGTCTGTGATATGTTGAC-1-Inf_J6,Inf_J6,3147,3147,8.054523,7126.0,8.871646,22.537188,0.266629,0.0,0.0
TTTGGCGGTTGGATGAATGTTGAC-1-Inf_J6,Inf_J6,4975,4975,8.512382,15833.0,9.669915,15.208741,0.025264,0.0,0.0


In [ ]:
#scDbFinder

In [ ]:
for idx, adata in enumerate(adatas):

    counts = np.array(adata.X) 
    print(counts.shape)


    ro.r.assign("counts_data", counts)
    ro.r(f"sce_{idx} <- SingleCellExperiment(list(counts=counts_data))")
    ro.r(f"set.seed(123)")
    ro.r(f"sce_{idx} <- scDblFinder(sce_{idx})")

    ro.r(f"doublet_score_{idx} <- sce_{idx}$scDblFinder.score")
    ro.r(f"doublet_class_{idx} <- sce_{idx}$scDblFinder.class")
    
    # Extraire les résultats en Python si nécessaire
    doublet_score = ro.r(f"doublet_score_{idx}")
    doublet_class = ro.r(f"doublet_class_{idx}")

In [ ]:
print(adatas[0].X.shape)
print(type(adatas[0].X))

In [23]:
def mad_outlier(adata, metric, nmads, upper_only = False):
    M = adata.obs[metric]
    
    if not upper_only:
        return (M < np.median(M) - nmads * mad(M)) | (M > np.median(M) + nmads * mad(M))
    
    return (M > np.median(M) + nmads * mad(M))

In [24]:
clf = doubletdetection.BoostClassifier(
    n_iters=10,
    clustering_algorithm="louvain",
    standard_scaling=True,
    pseudocount=0.1,
    n_jobs=-1)

In [25]:
def pp(adata):
    adata = adata[adata.obs.pct_counts_mt < 6] 
    
    bool_vector = mad_outlier(adata, 'log1p_total_counts', 5) +\
            mad_outlier(adata, 'log1p_n_genes_by_counts', 5) +\
            mad_outlier(adata, 'pct_counts_in_top_20_genes', 5) +\
            mad_outlier(adata, 'pct_counts_mt', 3, upper_only = True)
    adata = adata[~bool_vector]

    adata.uns['cells_removed'] = sum(bool_vector)

    doublets = clf.fit(adata.X).predict(p_thresh=1e-16, voter_thresh=0.6)
    doublet_score = clf.doublet_score()

    adata.obs["doublet_dbd"] = doublets
    adata.obs["doublet_score_dbd"] = doublet_score

    adata.uns['doublets_removed_dbd'] = adata.obs.doublet_dbd.sum()
    adata = adata[adata.obs.doublet_dbd == 0]
    #adata = adata[adata.obs.scDblFinder_class == 0]

    return adata

In [26]:
adatas = [pp(ad) for ad in adatas]

  0%|          | 0/10 [00:00<?, ?it/s]

AttributeError: 'csr_matrix' object has no attribute 'A'

In [ ]:
# partie avec choix de la prédiction en fonction d'une méthode machine learning ou pondérée

In [ ]:
for adata in adatas:
    print(len(adata), adata.uns['cells_removed'], adata.uns['doublets_removed_dbd'])

In [ ]:
os.makedirs('C:/Users/test/Documents/yohan/Data processing pipeline/scRNA/Output/processed_data', exist_ok = True)

In [ ]:
for ad in adatas:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join("processed_data", f'adata_{unique_id}.h5ad')
    ad.write(file_path)
    print(f"Saved: {file_path}")